In [ ]:
#pre-trained model to run food-11 dataset
# 11 classes, 3 splits: train, validation, evaluation
#use train, validation for training phase's train/test set
#use evaluation for testing model

In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
(train_ds,test_ds), ds_info = tfds.load(
    'food101',
    split=['train','validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

print(ds_info)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/75750 [00:00<?, ? examples/s]

In [ ]:
fig = tfds.show_examples(train_ds,ds_info,rows=4,cols=4)

def normalize_img(image,label):
    return tf.cast(image,tf.float32)/255.0,label
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.map(normalize_img, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.cache()
train_ds = train_ds.shuffle(ds_info.splits['train'].num_examples)
train_ds = train_ds.batch(64)
train_ds = train_ds.prefetch(AUTOTUNE)

test_ds = test_ds.map(normalize_img, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.cache()
test_ds = test_ds.batch(128)
test_ds = test_ds.prefetch(AUTOTUNE)

In [ ]:
train_ds

In [ ]:
vgg16 = keras.applications.vgg16
conv_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
for layer in conv_model.layers: 
    layer.trainable = False
x = keras.layers.Flatten()(conv_model.output)
x = keras.layers.Dense(101, activation='relu')(x)
x = keras.layers.Dense(101, activation='relu')(x)
x = keras.layers.Dense(101, activation='relu')(x)
predictions = keras.layers.Dense(1, activation='softmax')(x)
full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
full_model.summary()

In [ ]:
full_model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adamax(lr=0.001),
                  metrics=['acc'])
checkpointer = ModelCheckpoint(filepath='./best_model.hdf5', verbose=1, save_best_only=True)
history = full_model.fit(
    train_ds, 
    validation_data = test_ds,
    epochs=5,
    callbacks=[checkpointer],
)

full_model.save('./model_trained.hdf5')